In [242]:
import re
import time
import pandas as pd
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Click the "Load More" button for scraping more news

In [243]:
def extendPage():
    while True:
        try:
            try:
                # Just to locate to the Load More button to be clicked
                load_more = driver.find_element_by_class_name('quicklinks')
                actions = ActionChains(driver)
                actions.move_to_element(load_more).perform()
            except NoSuchElementException:
                print('Skip')
                break
            except StaleElementReferenceException:
                print('Skip')
                break

            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "loadMorestories"))).click()
            break
        except ElementClickInterceptedException:
            print("Try again")
            pass
        except TimeoutException:
            print("Try again")
            pass
        except NoSuchElementException:
            print("Try again")
            pass

### Iterate the given list to text or url

In [244]:
def toList(given_list, condition):
    convert = []
    for i in given_list:
        if condition == 'link':
            convert.append(i.get_attribute('href'))
        else:
            convert.append(i.text)
    non_empty_ele = [j for j in convert if j]
    return non_empty_ele

### Activate chrome driver

In [245]:
driver = webdriver.Chrome()
driver.get('https://www.thestar.com.my/')
time.sleep(10)

### Extract all types of news

In [246]:
types = driver.find_elements_by_xpath(".//li[@class = 'dropdown']")
all_type = toList(types, 'text')
all_type.remove('Videos')
all_type.remove('Photos')
print(all_type)

['StarPlus', 'News', 'Asean+', 'Business', 'Sport', 'Metro', 'Lifestyle', 'Food', 'Tech', 'Education', 'Opinion', 'StarPicks']


In [247]:
links_list = []
news_list = []

### Browse through every type of webpage, scrape the links and store into a list


In [248]:
for element in all_type:
    xpath = f"//a[contains(text(), '{element}')]"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))).click()
    print(xpath)
    extendPage()

    travel1 = driver.find_elements_by_xpath(".//main//div[5]//h2/a")
    if element.lower() == 'starplus' or element.lower() == 'business' or element.lower() == 'lifestyle' or element.lower() == 'opinion':
        travel2 = driver.find_elements_by_xpath(".//main//div[6]//h2/a")
    else:
        travel2 = driver.find_elements_by_xpath(".//main//div[6]/div/div/div[2]//h2/a")

    links_list1 = toList(travel1, 'link')
    links_list2 = toList(travel2, 'link')
    links_list = links_list + links_list1 + links_list2
    print(len(links_list))

driver.close()
driver.quit()

//a[contains(text(), 'StarPlus')]
Skip
27
//a[contains(text(), 'News')]
76
//a[contains(text(), 'Asean+')]
Skip
115
//a[contains(text(), 'Business')]
165
//a[contains(text(), 'Sport')]
214
//a[contains(text(), 'Metro')]
263
//a[contains(text(), 'Lifestyle')]
Skip
305
//a[contains(text(), 'Food')]
344
//a[contains(text(), 'Tech')]
Skip
383
//a[contains(text(), 'Education')]
Skip
393
//a[contains(text(), 'Opinion')]
Skip
420
//a[contains(text(), 'StarPicks')]
Skip
437


### Remove duplicates from list


In [249]:
filtered_links = []
[filtered_links.append(x) for x in links_list if x not in filtered_links]
print(f'No. of non-duplicated link: {len(filtered_links)}')

No. of non-duplicated link: 298


### Considering the Food category contains some recipes which will lead to another web page, therefore I decided to remove non The Star link here


In [250]:
for index in range(5):
    for i, ele in enumerate(filtered_links):
        if not ele.startswith('https://www.thestar.com.my/'):
            del filtered_links[i]
print(f'No. of non-TheStar link: {len(filtered_links)}')
filtered_links

No. of non-TheStar link: 296


['https://www.thestar.com.my/news/nation/2021/09/16/interactive-the-bahasa-in-malaysia',
 'https://www.thestar.com.my/news/nation/2021/08/27/interactive-over-two-million-students-eligible-for-vaccination',
 'https://www.thestar.com.my/starpicks/2021/09/12/committed-to-changing-lives?utm_medium=thestar&utm_source=rosnawidget&utm_campaign=Coway+120921',
 'https://www.thestar.com.my/news/nation/2021/09/09/interactive-from-the-desks-of-our-journalists',
 'https://www.thestar.com.my/news/nation/2021/09/02/interactive-how-to-boost-vaccinations-in-malaysia039s-six-target-states',
 'https://www.thestar.com.my/news/nation/2021/09/02/interactive-how-much-are-you-at-risk-from-accidents-at-work',
 'https://www.thestar.com.my/news/nation/2021/08/19/interactive-hospital-admissions-and-icu-cases-fall-in-some-states-as-vaccinations-rise',
 'https://www.thestar.com.my/news/nation/2021/08/10/unvaccinated-form-bulk-of-malaysia039s-new-covid-19-cases',
 'https://www.thestar.com.my/news/nation/2021/08/05/i

### Retrieve yesterday news

In [251]:
# Get yesterday date
yesterday = datetime.now() - timedelta(1)
date_required = f'{yesterday.year}/{yesterday.month:02d}/{yesterday.day:02d}'

yesterday_links = []
[yesterday_links.append(y) for y in filtered_links if re.search(date_required, y)]

print(f'No. of non-TheStar link: {len(yesterday_links)}')
yesterday_links

No. of non-TheStar link: 142


['https://www.thestar.com.my/starpicks/2021/09/18/mark-your-calendar-for-huawei-wearable-and-audio-day?utm_medium=thestar&utm_source=rosnawidget&utm_campaign=20210918_Huawei',
 'https://www.thestar.com.my/news/world/2021/09/18/mexican-president-pitches-new-regional-block-similar-to-european-union',
 'https://www.thestar.com.my/news/nation/2021/09/18/sabah-govt-announces-further-relaxation-of-activities-for-fully-vaccinated-from-monday-sept-20',
 'https://www.thestar.com.my/news/nation/2021/09/18/selangor-police-issue-423-compounds-for-disregarding-sop',
 'https://www.thestar.com.my/news/nation/2021/09/18/covid-19-army039s-greater-klang-valley-task-force-ceases-operation-says-hishammuddin',
 'https://www.thestar.com.my/news/nation/2021/09/18/over-2500-people-entering-langkawi-screened-one-tested-positive-for-covid-19-says-health-dg',
 'https://www.thestar.com.my/news/nation/2021/09/18/man-found-drowned-while-trying-to-save-friend-in-kuala-kubu-bharu',
 'https://www.thestar.com.my/news/w

In [252]:
new_news = []
try:
    with open('checking.txt', 'r') as f:
        checking_links = f.read().splitlines()
        
    if checking_links[0] == date_required:
        with open('checking.txt', 'a') as f:
            for url1 in yesterday_links:
                for index, url2 in enumerate(checking_links):
                    if url1 == url2:
                        break
                    elif index == len(checking_links) - 1:
                        new_news.append(url1)
                        f.write(url1 + '\n')
    else:
        with open('checking.txt', 'w') as f:
            f.write(date_required + '\n')
            for url in yesterday_links:
                new_news.append(url)
                f.write(url + '\n')  

except FileNotFoundError:
    with open('checking.txt', 'w') as f:
        f.write(date_required + '\n')
        for url in yesterday_links:
            new_news.append(url)
            f.write(url + '\n')       
                    
print(f'Links to crawl: {len(new_news)}')
new_news

Links to crawl: 142


['https://www.thestar.com.my/starpicks/2021/09/18/mark-your-calendar-for-huawei-wearable-and-audio-day?utm_medium=thestar&utm_source=rosnawidget&utm_campaign=20210918_Huawei',
 'https://www.thestar.com.my/news/world/2021/09/18/mexican-president-pitches-new-regional-block-similar-to-european-union',
 'https://www.thestar.com.my/news/nation/2021/09/18/sabah-govt-announces-further-relaxation-of-activities-for-fully-vaccinated-from-monday-sept-20',
 'https://www.thestar.com.my/news/nation/2021/09/18/selangor-police-issue-423-compounds-for-disregarding-sop',
 'https://www.thestar.com.my/news/nation/2021/09/18/covid-19-army039s-greater-klang-valley-task-force-ceases-operation-says-hishammuddin',
 'https://www.thestar.com.my/news/nation/2021/09/18/over-2500-people-entering-langkawi-screened-one-tested-positive-for-covid-19-says-health-dg',
 'https://www.thestar.com.my/news/nation/2021/09/18/man-found-drowned-while-trying-to-save-friend-in-kuala-kubu-bharu',
 'https://www.thestar.com.my/news/w

### Browse through each url and crawl the news content

In [253]:
for link in new_news:
    driver = webdriver.Chrome()
    url = f"{link}"
    print(link)
    driver.get(url)

    try:
        title = driver.find_element_by_tag_name('h1').text
    except NoSuchElementException:
        driver.close()
        driver.quit()
        continue

    try:
        author = driver.find_elements_by_xpath(".//div[@class = 'credit__authors']/a")
        author = toList(author, 'text')
        author = re.sub(",", " ", '，'.join(author))
    except NoSuchElementException:
        author = ''
        pass

    try:
        date = driver.find_element_by_xpath(".//p[@class = 'date']").text
    except NoSuchElementException:
        date = ''
        pass

    try:
        timestamp = driver.find_element_by_xpath(".//time[@class = 'timestamp']").text
    except NoSuchElementException:
        timestamp = ''
        pass

    # Extract all the text of the news content
    parent = driver.find_element_by_id('story-body')
    content = list(parent.text.split('\n'))

    # The following 4 statements are to extract all irrelevant content such as chart and image caption
    chart_text = parent.find_elements_by_xpath(".//div[contains(@class, 'flourish')]")
    inline_text = parent.find_elements_by_xpath(".//span[contains(@class, 'inline-caption')]")
    caption_text = parent.find_elements_by_xpath(".//p[contains(@class, 'caption')]")
    extra_text = chart_text + inline_text + caption_text

    full_content = toList(extra_text, 'text')
    full_content = list(set(full_content))

    # Filter out irrelevant content
    for index, i in enumerate(content):
        for j in full_content:
            if i == j:
                content[index] = ''

    # Remove the enter and empty field from the extracted news content
    real_content = ''
    for element in content:
        real_content += element

    # Determine the type of the news by using the url
    for category in all_type:
        string = f"https://www.thestar.com.my/{category.lower()}"
        result = re.search(string, link)
        if result is not None:
            break

    print(date)
    print(timestamp)
    print(title)
    print(category)
    print(author)
    print(real_content)

    news_items = {'Date': date, 'Time': timestamp, 'Type': category, 'Author': author, 'Title': title, 'Content': real_content}
    news_list.append(news_items)
    print(len(news_list))
    driver.close()
    driver.quit()
    time.sleep(3)

https://www.thestar.com.my/starpicks/2021/09/18/mark-your-calendar-for-huawei-wearable-and-audio-day?utm_medium=thestar&utm_source=rosnawidget&utm_campaign=20210918_Huawei
Saturday, 18 Sep 2021
9:00 AM MYT
Mark your calendar for HUAWEI Wearable and Audio Day
StarPicks

WORKING out at home has become a part of the new normal for some, with the goals to keep fit or change your appearance – ‘glow up’ – before returning to work or school.To gear up and for a healthier you, all while staying at home, HUAWEI will be launching its Wearable and Audio Day on Sept 25 for better prices on your gear, such as the HUAWEI Band 6 and HUAWEI FreeBuds 4i.Fret not, you can also choose from a whole host of other HUAWEI wearable and audio products on the HUAWEI Wearable and Audio Day.Valuable combo to keep activeWith a promotional price of RM419, the HUAWEI Band 6 and HUAWEI FreeBuds 4i combo might be your most valuable combo to stay safe and active.When you are putting all your effort into working out, le

https://www.thestar.com.my/news/nation/2021/09/18/selangor-police-issue-423-compounds-for-disregarding-sop
Saturday, 18 Sep 2021
9:04 PM MYT
Selangor police issue 423 compounds for disregarding SOP
News

SEPANG (Bernama): Selangor police have issued 423 compounds to members of the public for flouting the standard operating procedure (SOP) under Phase Two of the National Recovery Plan (NRP) from Sept 10 until Friday (Sept 17).Selangor police chief, Datuk Arjunaidi Mohamed, said the compounds were issued for various offences like failure to wear a mask, disregarding social distancing and others.He said about 3,000 personnel from 700 monitoring teams have been assigned to ensure that the public adhere to the SOP since Klang Valley transitioned to Phase Two of the NRP, where inter district travelling is allowed and many had taken advantage of this to do various recreational activities."We are monitoring the beaches and places of attractions in Selangor like waterfalls to ensure visitors cl

https://www.thestar.com.my/news/nation/2021/09/18/man-found-drowned-while-trying-to-save-friend-in-kuala-kubu-bharu
Saturday, 18 Sep 2021
8:04 PM MYT
Man found drowned while trying to save friend in Kuala Kubu Bharu
News

KUALA LUMPUR (Bernama): A man who attempted to help a friend who had slipped when taking a selfie in a dam area at the Kampung Pertak bridge, in Kuala Kubu Bharu, Selangor on Saturday (Sept 18) afternoon had been found drowned.Selangor Fire and Rescue Department director Norazam Khamis said the body of the 19-year-old victim was found at about 6pm on Saturday evening.Earlier, the department received a call at 3.54pm before firefighters from the Kuala Kubu Bharu Fire and Rescue Station rushed to the scene.He said the 19-year-old victim was trying to save his friend who had slipped off the bridge while taking a selfie."His 20-year-old friend managed to swim to safety before the fire brigade arrived at the location, but the victim who jumped in to save him did not reappe

https://www.thestar.com.my/news/nation/2021/09/18/another-seven-month-extension-for-learners-driver-licence-says-dr-wee
Saturday, 18 Sep 2021
4:06 PM MYT
Another seven-month extension for learner’s driver licence, says Dr Wee
News
VENESA DEVI
YONG PENG: The Road Transport Department (JPJ) has granted another seven months extension to learner’s driving licence (LDL).Transport Minister Datuk Seri Dr Wee Ka Siong said that this would help those who are unable to go through their driving test due to the Covid-19 pandemic."Previously, we extended the validity of the LDL, which was only three months, to November this year."We have been receiving feedback and calls from younger people holding LDL about the expiry of the license."As the Transport Minister, I have agreed to give an extension for the LDL to June 30 next year," he said.He said this when met after visiting a JPJ mobile counter here in Kampung Kangkar Baru on Saturday (Sept 18)."Those with LDL no longer have to rush to go through t

https://www.thestar.com.my/news/world/2021/09/18/australia-made-039huge039-mistake039-cancelling-submarine-deal-says-french-ambassador
Saturday, 18 Sep 2021
6:49 PM MYT
Australia made 'huge' mistake' cancelling submarine deal, says French ambassador
News
LIDIA KELLY，STEFICA NICOL BIKES
MELBOURNE (Reuters) - Australia has made a "huge" diplomatic error, the French ambassador said on Saturday having been recalled to Paris after Canberra ditched a multi billion dollar order for French submarines in favour of an alternative deal with the United States and Britain."I think this has been a huge mistake, a very, very bad handling of the partnership – because it wasn't a contract, it was a partnership that was supposed to be based on trust, mutual understanding and sincerity," Ambassador Jean-Pierre Thebault told journalists in Canberra.Australia said on Thursday it would scrap the deal signed in 2016 for France's Naval Group to build a fleet of conventional submarines and would instead build 

...

https://www.thestar.com.my/lifestyle/entertainment/2021/09/18/seungkwan-from-k-pop039s-seventeen-exchanges-microphone-for-badminton-racket-in-new-variety-show
Saturday, 18 Sep 2021
12:00 PM MYT
Seventeen's Seungkwan learns to play badminton in new variety show 'Racket Boys'
Lifestyle
CALLEN JAMES MARSH
Seungkwan from the K-pop boy band Seventeen has found a new challenge in life – badminton.It was announced on Sept 16 that Seungkwan will be joining the cast of tvN's new variety show, Racket Boys. Other celebrities joining the show include Oh Sang Wook, the Korean fencer who recently won a gold medal for his country in the Tokyo Olympics and 14-year-old actor Jung Dong-won. More celebrities will be recruited by the station soon.The variety show revolves around the challenge of learning how to play badminton. And it is not just fun and games. The participants will have to take the sport seriously as they will eventually participate in a national competition.Seungkwan may have an edge ove

https://www.thestar.com.my/lifestyle/health/2021/09/18/plant-ing-that-zen-feeling-around-your-home
Saturday, 18 Sep 2021
8:00 AM MYT
‘Plant’-ing that zen feeling around your home
Lifestyle
AGENCY
In your arsenal of tools and tactics for staying calm and improving your well-being, plants can prove to be precious resources.Whether in herbal tea, decorative planters or essential oils, they can help us relieve tension and soothe the mind.Here’s how to best use plants to improve your mental health:> Dried flowers for rooms and closetsWhen it comes to scents, lavender is high on the list for a calming effect.It can be placed in the entrance of rooms or your home for a sensory atmosphere that imbues a peaceful vibe as soon as you walk in the door.Balconies or windows are also great spots to place them.And if you really don’t have the space, you can slip some into small fabric bags and tuck them into a closet or drawer.Rosemary and thyme can also be used in this way, or for scenting the kitche

https://www.thestar.com.my/tech/tech-news/2021/09/18/you-had-one-job-google039s-alarm-fails-countless-users-after-update
Saturday, 18 Sep 2021
7:00 AM MYT
You had one job: Google's alarm fails countless users after update
Tech

Hundreds of Android smartphone owners have been venting their anger at Google after the tech giant's Clock app failed to activate its alarms following an update in late August.The Clock app, among the world's most widely used alarm apps with more than a billion downloads, is thought to have left many of its users snoozing in recent weeks due to a bug that sees the app remain silent and instead show a notification of a "missed" alarm.Google has since solved the problem with a Sept 7 update to the Clock app, according to the Google news website Androidpolice.com."The alarm didn't do anything. No vibration, no sound. Missed the alarm. Now I need a new job," reads one of hundreds of angry reviews on the Google Play Store since the start of September. "Thanks for mak

https://www.thestar.com.my/tech/tech-news/2021/09/18/verizon-sweetens-subsidies-on-iphones-to-match-competition
Saturday, 18 Sep 2021
2:29 AM MYT
Verizon sweetens subsidies on iPhones to match competition
Tech

(Reuters) - Verizon Communications said on Friday it is increasing its subsidies on offer for Apple Inc's newly launched iPhone 13, in a bid to attract more customers to the U.S. carrier's network.Verizon said it would now offer trade-in subsidies of up to $1000, matching rival AT&T Inc, from $700 previously. The biggest subsidies will be given to customers who turn in the iPhone 11 and iPhone 12 models.Apple's new range of iPhones come at an unchanged price, but carriers are using aggressive subsidies as a way to hold on to existing customers and attract new ones.The phones will also have custom 5G antennas and radio components for faster speeds. This push towards expanding 5G connectivity comes as telecom firms in the United States race to bolster their 5G infrastructure.The i

https://www.thestar.com.my/starpicks/2021/09/18/time-travel-to-the-70s
Saturday, 18 Sep 2021
8:51 AM MYT
TIME TRAVEL TO THE 70S
StarPicks

Swiss watchmaker brings back disco with its retro-aesthetic timepieces.LUXURY watchmaker Tissot has a long history of sharing its craft through stories. Since its beginning in 1853, it has woven the evolution of its era into its timepieces.A throwback to the 1970s, Tissot’s upgraded PRX Automatic reintroduces a sophisticated and young-at-heart piece with on-trend retro-aesthetics.This essential timepiece has an integrated case and bracelet, which is made remarkable by its slim, timeless and solid design – an ideal watch for those with a passion for design and an eye for ingenuity.The upgraded PRX Automatic – that subtly incorporates the style and design of the 70s – comes with a Powermatic 80 movement and an even edgier touch of style than its quartz predecessor.Deeply rooted in today’s design cues of the elegant sportswatch, this nod to the disco e

### Store the crawled data into data frame

In [254]:
data = pd.DataFrame(news_list)
data = data.sort_values('Time')
data.head()

,Date,Time,Type,Author,Title,Content
70,"Saturday, 18 Sep 2021",,Business,THOMAS HUONG,Brighter outlook for Malaysia’s oil and gas se...,RISING global oil prices amidst increasing Cov...
79,"Saturday, 18 Sep 2021",,Business,,Gadang’s subsidiary bags RM100mil deal,KUALA LUMPUR: Gadang Holdings Bhd’s wholly-own...
78,"Saturday, 18 Sep 2021",,Business,,Early economic resumption vital,THE rising number of Covid-19 cases since earl...
77,"Saturday, 18 Sep 2021",,Business,,Fundaztic to fund solar initiatives,"PEOPLENDER Sdn Bhd, which operates peer-to-pee..."
76,"Saturday, 18 Sep 2021",,Business,B K SIDHU,CEOs optimistic of recovery,IT has been a tough 18 months since the Covid-...


In [255]:
print("Shape    : ", data.shape)

Shape    :  (142, 6)


### Store news to csv file

In [256]:
data.to_csv('News.csv', mode = 'a', index = False, header = False)

In [257]:
df = pd.read_csv('News.csv')
print("Shape    : ", df.shape)

Shape    :  (324, 6)


<hr>

### Combine 2 data frames

In [258]:
# # Concatenate current and older data
# data = pd.concat([before, current])
# # Drop duplicates between previous and current data
# data.drop_duplicates(inplace = True, subset = ['Content'])
# # Store the data back to the News csv file
# data.to_csv('News.csv', index=False)
# data.head()

In [259]:
# print("Rows     : ", data.shape[0])
# print("Columns  : ", data.shape[1])
# print("Shape    : ", data.shape)
# print("Features : ", data.columns.tolist())